In [ ]:
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#8F003C','#eb3446','Tourney','Smokum',33,10
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h4 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h4>"""%string))
    
    
dhtml('Exploring Impact of COVID-19 on Digital Learning' )

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.display import HTML
HTML("""
<style>
h1,h2,h3 {
	margin: 1em 0 0.5em 0;
	font-weight: 600;
	font-family: 'Titillium Web', sans-serif;
	position: relative;  
	font-size: 36px;
	line-height: 40px;
	padding: 15px 15px 15px 2.5%;
	color: #00018D;
	box-shadow: 
		inset 0 0 0 1px rgba(97,0,45, 1), 
		inset 0 0 5px rgba(53,86,129, 1),
		inset -285px 0 35px #F2D8FF;
	border-radius: 0 10px 0 15px;
	background: #FFD8B2
    
}
</style>
""")

#### Disclaimer: This is just a starter Notebook. I will update this notebook regularly. Till then Stay Tuned and Take Care. 

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">✅ Importing Required Libraries</h1>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objs as go
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from plotly import tools
import re
from plotly.offline import init_notebook_mode, plot, iplot
from wordcloud import WordCloud, STOPWORDS 
from warnings import filterwarnings
filterwarnings('ignore')
import missingno as msno
import glob

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">🚚 Loading the Dataset:</h1>

In [ ]:
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    district_id = filename.split("/")[4].split(".")[0]
    df["district_id"] = district_id
    li.append(df)
    
engagement_df = pd.concat(li)
engagement_df = engagement_df.reset_index(drop=True)


In [ ]:
engagement_df.head(15).style.set_caption("Engagement Dataframe").set_properties(**{'background-color': 'peachpuff',
                           'color': 'midnightblue','border': '1.5px  solid black'})

In [ ]:
districts_df = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
districts_df.head(15).style.set_caption("Districts Dataframe").set_properties(**{'background-color': 'peachpuff',
                           'color': 'midnightblue','border': '1.5px solid black'})

In [ ]:
products_df = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
products_df.head(15).style.set_caption("Products Dataframe").set_properties(**{'background-color': 'peachpuff',
                           'color': 'midnightblue','border': '1.5px solid black'})

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">🔍 Basic Exploration:</h1>

In [ ]:
products_df.info()

In [ ]:
print('⁕'*50)
print("Null Value in Engagement Dataframe:")
print('⁕'*50)
print(engagement_df.isnull().sum())
print("\n")
print('⁕'*50)
print("Null Value in Districts Dataframe:")
print('⁕'*50)
print(districts_df.isnull().sum())
print("\n")
print('⁕'*50)
print("Null Value in Products Dataframe:")
print('⁕'*50)
print(products_df.isnull().sum())


In [ ]:
print("District Dataframe Columns:")
print('⁕'*50)
print(districts_df.columns)
print("\n")
print("Engagement Dataframe Columns:")
print('⁕'*50)
print(engagement_df.columns)
print("\n")
print("Products Dataframe Columns:")
print('⁕'*50)
print(products_df.columns)

In [ ]:
print('⁕'*50)
print("Shape for Districts Dataframe:",districts_df.shape)
print("Shape for Engagement Dataframe:",engagement_df.shape)
print("Shape for Products Dataframe:",products_df.shape)
print('⁕'*50)

In [ ]:
msno.bar(products_df, figsize=(10,5), fontsize=12)
plt.show()

In [ ]:
msno.bar(engagement_df, figsize=(10,5), fontsize=12)
plt.show()

In [ ]:
msno.bar(districts_df, figsize=(10,5), fontsize=12)
plt.show()

In [ ]:
print('⁕'*50)
print("Number of Duplicate value in Districts Dataframe:",districts_df.duplicated().sum())
print("Number of Duplicate value in Engagement Dataframe:",engagement_df.duplicated().sum())
print("Number of Duplicate value in Products Dataframe:",products_df.duplicated().sum())
print('⁕'*50)

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">🌍 Some Geo-Analysis:</h1>

In [ ]:
locations=pd.DataFrame({"Name":districts_df['state'].unique()})

In [ ]:
geolocator=Nominatim(user_agent="app")
lat=[]
lon=[]
for location in locations['Name']:
    location = geolocator.geocode(location)    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)

In [ ]:
locations['lat']=lat
locations['lon']=lon

In [ ]:
Rest_locations=pd.DataFrame(districts_df['state'].value_counts().reset_index())

In [ ]:
Rest_locations.columns=['Name','count']
final_loc=Rest_locations.merge(locations,on='Name',how="left").dropna()
final_loc.head(15).style.set_caption("Locations Dataframe").set_properties(**{'background-color': 'peachpuff',
                           'color': 'midnightblue','border': '1.5px solid black'})

In [ ]:
def generateBaseMap(default_location=[37.0902, -95.7129], default_zoom_start=4):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

In [ ]:
basemap=generateBaseMap()

In [ ]:
HeatMap(final_loc[['lat','lon','count']],zoom=20,radius=20).add_to(basemap)

In [ ]:
basemap

In [ ]:
FastMarkerCluster(data=final_loc[['lat','lon','count']].values.tolist()).add_to(basemap)

basemap

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">🍩 Some Donut Charts:</h1>

In [ ]:
labels = list(districts_df.locale.value_counts().index)
values = districts_df['state'].value_counts()
# colors = ['mediumslateblue', 'darkorange']
fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=20,
                  marker=dict( line=dict(color='#000000', width=3)))
fig.update_layout(title="Locale Distribution ",
                  titlefont={'size': 30},      
                  )
fig.show()

In [ ]:
labels = list(districts_df.state.value_counts().index)
values = districts_df['state'].value_counts()
# colors = ['mediumslateblue', 'darkorange']
fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=20,
                  marker=dict( line=dict(color='#000000', width=3)))
fig.update_layout(title="State Distribution ",
                  titlefont={'size': 30},      
                  )
fig.show()

In [ ]:
labels = list(products_df['Sector(s)'].value_counts().index)
values = products_df['Sector(s)'].value_counts()
# colors = ['mediumslateblue', 'darkorange']
fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=20,
                  marker=dict( line=dict(color='#000000', width=3)))
fig.update_layout(title="Sector Distribution ",
                  titlefont={'size': 30},      
                  )
fig.show()

<h1 style="background-color:#FF85A3;font-size:20px;color:#00033E;font-weight : bold">☁ Basic Word Clouds:</h1>

In [ ]:
cloud = WordCloud(width=1440, height=1080,stopwords={'nan'}).generate(" ".join(districts_df['state'].astype(str)))
plt.figure(figsize=(15, 10))
plt.imshow(cloud)
plt.axis('off')

In [ ]:
cloud = WordCloud(width=1440, height=1080,stopwords={'nan'}).generate(" ".join(products_df['Product Name'].astype(str)))
plt.figure(figsize=(15, 10))
plt.imshow(cloud)
plt.axis('off')

In [ ]:
cloud = WordCloud(width=1440, height=1080,stopwords={'nan'}).generate(" ".join(products_df['Provider/Company Name'].astype(str)))
plt.figure(figsize=(15, 10))
plt.imshow(cloud)
plt.axis('off')

In [ ]:
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#000CA4','#eb3446','Big Shoulders Inline Text','Smokum',50,10
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h4 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h4>"""%string))
    
    
dhtml(' Do UPVOTE if you like my work ' )

![](https://capoprogetto.it/wp-content/uploads/2020/12/wip-work-in-progress-1024x701.jpg)